In [1]:
import numpy as np
import qiskit 
from mgbenchmark.main import mg_unitary_to_so, generate_jw_list, generate_jw_basis, unitary_to_superoperator, compound_matrix, mg_so_to_superoperator
from mgbenchmark.utils import generate_binary_strings, superop_to_dictionaries, dictionary_to_distribution, string_to_pauli, generate_jw_list_pauli
from qiskit import QuantumCircuit, Aer, execute
from qiskit.extensions import UnitaryGate
from qiskit.providers.aer import AerSimulator
from qiskit_aer.noise import (NoiseModel, QuantumError, ReadoutError,
    pauli_error, depolarizing_error, thermal_relaxation_error)
from qiskit.tools.visualization import array_to_latex
from qiskit.quantum_info import Kraus, SuperOp, Chi
import random

## Matchgate Benchmarking

Below is a demo of Algorithm 1 in https://arxiv.org/abs/2404.07974.

We start off by defining the matchgate which we will test with our circuit. Here's an example:

In [3]:
theta = 0.12345
def gate_matrix(angle):
    matrix = np.array([[np.exp(1j * angle / 2), 0, 0, 0],
                       [0, np.exp(1j * angle / 2), 0, 0],
                       [0, 0, np.exp(-1j * angle / 2), 0],
                       [0, 0, 0, np.exp(-1j * angle / 2)]])
    return matrix

unitary = gate_matrix(-theta)

array_to_latex(unitary, prefix="U = ")

<IPython.core.display.Latex object>

We can define some other more complicated unitaries to use in the algorithm as well:

In [4]:
theta = 1.2
phi = 0.75

clifford1 = np.asarray([[1, 0, 0, 0],
                        [0, 1, 0, 0],
                        [0, 0, 0, 1],
                        [0, 0, 1, 0]])

clifford2 = np.asarray([[0, 0, 1, 0],
                        [0, 0, 0, -1],
                        [1, 0, 0, 0],
                        [0, -1, 0, 0]])

fsim = np.asarray([[1, 0, 0, 0],
                     [0, np.cos(theta), -1j*np.sin(theta), 0],
                     [0, -1j*np.sin(theta), np.cos(theta), 0],
                     [0, 0, 0, np.exp(1j * phi)]])

mg1 = np.asarray([[1, 0, 0, 0],
                     [0, np.cos(theta), -np.sin(theta), 0],
                     [0, np.sin(theta), np.cos(theta), 0],
                     [0, 0, 0, 1]])

mg2 = np.asarray([[1, 0, 0, 0],
                     [0, np.cos(2*theta), -1j*np.sin(2*theta), 0],
                     [0, -1j*np.sin(2*theta), np.cos(2*theta), 0],
                     [0, 0, 0, 1]])
mg3 = np.asarray([[1, 0, 0, 0],
                     [0, np.cos(-0.5*theta), -np.sin(-0.5*theta), 0],
                     [0, np.sin(-0.5*theta), np.cos(-0.5*theta), 0],
                     [0, 0, 0, 1]])


Next, we calculate a superoperator matrix. We can do this the 'standard' way, by taking Hilbert-Schmidt inner products, or we can convert $U$ to an $SO(2n)$ matrix and take its compound matrices. Both methods are available (the latter performs better for large matrices).

In [5]:
#r = mg_unitary_to_so(unitary)
#superop = mg_so_to_superoperator(r)

superop = unitary_to_superoperator(unitary)

array_to_latex(superop, max_size=100)

<IPython.core.display.Latex object>

We can convert the superoperator matrix $ \ \hat{\mathcal{U}} \ $ into a series of dictionaries; one giving us the matrix elements, the other giving us the sampling probabilities. Then, we use the latter to form a probability distribution.

The algorithm for benchmarking consists of the following steps:
* Take the unitary $U$ and convert it to a superoperator matrix $ \ \hat{\mathcal{U}} \ $.
* Convert $ \ \hat{\mathcal{U}} \ $ to a series of dictionaries, one with probabilities and the other with non-zero superoperator matrix elements.
* Sample from the probability distribution to get an 'input' and 'output' operators $c_\mathbf{x}, c_\mathbf{y}$. In our paper, we index them using subsets $I \subseteq {1, 2, ..., 2n}$. Here, we index using bitstrings $\mathbf{x}$ such that $x_k = 1$ if $k \in I$, and $x_k = 0$ otherwise (this is equivalent & computationally easier to implement).
* Convert the bitstrings to Pauli operators and store the phase factors. 
* Prepare random eigenstates of the input state, evolve them with $U$, apply the gate error, and measure in the basis of the output operator. 
* Repeat the above step for a number of trials, and calculate the gate fidelity from the data.


We use qiskit's noise simulation capabilities to implement all-qubit depolarising noise as part of the simulation. We also randomly generate all of the circuits first, and then tally them up into jobs of multiple shots. This reduces the number of calls to the backend.

*Below is the MGC benchmarking algorithm, which returns the correct entanglement fidelity up to precision of $\pm 2 \epsilon$ with probability $1 - 2\delta$:*

In [7]:
# Specify the input unitary:
#matrix = np.eye(2)
#matrix = clifford1 @ mg2 @ mg1 @ mg3 @ clifford2
#matrix = fsim
#matrix = np.kron(mg2, np.eye(2)) @ np.kron(np.eye(2), mg1)
matrix = np.kron(mg2, np.eye(4)) @ np.kron(np.kron(np.eye(2), mg1), np.eye(2)) @ np.kron(np.eye(4), mg2)
#matrix = mg2 @ mg1 @ mg3 

# Specify key parameters:
err_prob = 0.1
epsilon = 0.1
delta = 0.1

#region main algorithm 

# Generate the superoperator and the dictionaries
superop = unitary_to_superoperator(matrix)
dictionary_mat, dictionary_prob = superop_to_dictionaries(superop)
indices, probs = dictionary_to_distribution(dictionary_prob)
ugate = UnitaryGate(matrix, label="U")

n = int(np.ceil(np.log2(len(matrix))))

shots_sum = 0
shots = 0
jw_list = generate_jw_list_pauli(n)

n_iter = int(np.ceil(1 / (delta * epsilon ** 2)))

jobs_dict = {}
circuit_dict = {}
phase_dict = {}

# Generate the circuits, assigning an identifier to each. 
# Prepare a dictionary tallying the number of times each circuit is sampled, 
# and another dictionary storing the circuit object itself.
# NOTE: Multiple dictionaries are used as the keys are not hashable, so we have 
# a unique hashable 'identifier' as keys and non-hashable objects as values.

for k in range(n_iter):

    # Input sampled binary string 
    s_pair = indices[np.random.choice(a=len(indices), size=None, replace=True, p=probs)]

    # Calculate the relevant matrix element
    r_element = dictionary_mat[s_pair]

    # Process the input string
    input_pauli = string_to_pauli(jw_list, s_pair[1])
    input_phase = input_pauli.phase
    input_pauli.phase = 0
    i_string = str(input_pauli)

    # Process the output string
    output_pauli = string_to_pauli(jw_list, s_pair[0]).adjoint()
    output_phase = output_pauli.phase
    output_pauli.phase = 0
    j_string = str(output_pauli)

    phase = (-1j) ** (input_phase + output_phase)

    n_sample = int(np.ceil(2 * np.log(2 / delta) / (n_iter * (epsilon * abs(r_element)) ** 2)))

    for m in range(n_sample):

        if s_pair[0] == "00"*n:
            shots_sum += 1
            shots += 1

        else:

            # Create the quantum circuit
            qci = QuantumCircuit(n,n)
            lamb = 1

            qci_identifier = s_pair
            qci_key = ""

            # INITIALISE THE CIRCUIT IN PAULI EIGENSTATE
            # Apply the cliffords to the circuit according to the input. 
            # This prepares an eigenstate of the pauli & stores the eigenvalue as lamb
            # NOTE: The indices are reversed due to the Qiskit Convention, so q_(n-1) represents the first qubit and q_0 the last 
            for i in range(len(i_string)):
                prob = random.randint(0,1)
                if prob == 0:
                    qci_key += "0"
                else:
                    qci_key += "1"

                if i_string[i] == 'I':
                    if prob == 0:
                        qci.id(n-1-i)
                    else: 
                        qci.x(n-1-i)
                elif i_string[i] == 'Z':
                    if prob == 0:
                        qci.x(n-1-i)
                        lamb = lamb * -1
                    else:
                        qci.id(n-1-i)
                elif i_string[i] == 'X':
                    if prob == 0:
                        qci.h(n-1-i)
                    else:
                        qci.x(n-1-i)
                        qci.h(n-1-i)
                        lamb = lamb * -1
                elif i_string[i] == 'Y':
                    if prob == 0:
                        qci.h(n-1-i)
                        qci.s(n-1-i)
                    else:
                        qci.x(n-1-i)
                        qci.h(n-1-i)
                        qci.s(n-1-i)
                        lamb = lamb * -1

            # APPEND THE UNITARY GATE TO INPUT STATE
            qci.barrier()
            qci.append(ugate, list(range(n)))
            qci.barrier()

            # MEASURE THE OUTPUT IN PAULI BASIS
            # Apply the cliffords to the circuit according to the output string. This will give measurements in the pauli basis
            for j in range(len(j_string)):
                if j_string[j] == "I" or j_string[j] == "Z":
                    qci.id(n-1-j)
                elif j_string[j] == "X":
                    qci.h(n-1-j)
                elif j_string[j] == "Y":
                    qci.sdg(n-1-j)
                    qci.h(n-1-j)

            for j in range(len(j_string)):
                if j_string[j] == 'X' or j_string[j] == 'Y' or j_string[j] == 'Z':
                    qci.measure(n-1-j, n-1-j)

            shots += 1

            qci_identifier += (qci_key,)

            if qci_identifier not in jobs_dict:
                jobs_dict[qci_identifier] = 1
                phase_dict[qci_identifier] = (lamb * phase) / (r_element * n_sample)
                circuit_dict[qci_identifier] = qci
            elif qci_identifier in jobs_dict:
                jobs_dict[qci_identifier] += 1

noise_model = NoiseModel()
dep_error = depolarizing_error(err_prob, n)
noise_model.add_quantum_error(dep_error, "U", list(range(n)))

# Execute the tallied circuits on the Aer simulator
for qci_identifier in jobs_dict:
    simulator = Aer.get_backend('aer_simulator')
    job = execute(circuit_dict[qci_identifier], backend=simulator, shots=jobs_dict[qci_identifier], noise_model=noise_model)
    result = job.result()
    counts = result.get_counts()
    for strings in counts:
        shots_sum += (-1)**strings.count('1') * phase_dict[qci_identifier] * counts[strings]

# Calculate the true entanglement fidelity from the top-left element of the noise process matrix.
processmat = Chi(dep_error).data / 2**(n)
true_entanglement_fidelity = processmat[0,0]

#endregion

print("entanglement fidelity:", np.real(shots_sum / n_iter), " | true entanglement fidelity:", np.real(true_entanglement_fidelity), " | total shots:", shots)

entanglement fidelity: 0.9283922565298481  | true entanglement fidelity: 0.9003906250000051  | total shots: 7404


The above code, modified to collect our data may be found in the `matchgate_benchmarking_data_collction.ipynb` notebook.